In [ ]:
import gradio as gr
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from PIL import Image
import numpy as np
import csv
import os
import random
from datetime import datetime

# Model and adapter paths
adapter_path = "/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1/checkpoint-155"

# Set up Qwen model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    cache_dir="/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1"
)

# Processor setup
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", 
    cache_dir="/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1", 
    max_pixels=720 * 28 * 28
)

# Load adapter
model.load_adapter(adapter_path)



In [ ]:
# Shelf life mapping based on fruit type and freshness
def get_shelf_life(fruit_type, freshness):
    shelf_life_map = {
        'apple': {
            'extreme freshness': 'around 7 days',
            'mild freshness': 'around 3-5 days',
            'worse': 'less than 1 days'
        },
        'banana': {
            'extreme freshness': 'around 4 days',
            'mild freshness': 'around 2-3 days',
            'worse': 'less than 2 days'
        },
        'potato': {
            'extreme freshness': 'around 45-60 days',
            'mild freshness': 'around 10-12 days',
            'worse': 'less than 3 days'
        },
        'orange': {
            'extreme freshness': 'around 7 days',
            'mild freshness': 'around 2-3 days',
            'worse': 'less than 1 days'
        },
        'guava': {
            'extreme freshness': 'around 10 days',
            'mild freshness': 'around 3-4 days',
            'worse': 'less than 2 days'
        },
        'pomegranate': {
            'extreme freshness': 'around 8 days',
            'mild freshness': 'around 3-4 days',
            'worse': 'less than 2 days'
        },
        'strawberry': {
            'extreme freshness': 'around 4 days',
            'mild freshness': 'around 2-3 days',
            'worse': 'less than 1 days'
        },
        # Add more fruits or vegetables here
    }

    fruit_type = fruit_type.lower()
    freshness = freshness.lower()

    if fruit_type in shelf_life_map and freshness in shelf_life_map[fruit_type]:
        return shelf_life_map[fruit_type][freshness]
    else:
        return "Unknown shelf life"

# Assign random freshness index based on freshness level
def get_freshness_index(freshness):
    if freshness == "extreme freshness":
        return random.randint(7, 9)
    elif freshness == "mild freshness":
        return random.randint(4, 6)
    elif freshness == "worse":
        return random.randint(1, 3)
    else:
        return -1

# Initialize CSV file
csv_file = "freshness_data.csv"
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Produce", "Freshness", "Freshness Index", "Estimated Shelf Life"])

# Function to process the image and assess freshness with shelf life
def process_freshness(image):
    # Convert numpy array to PIL Image
    pil_image = Image.fromarray(np.uint8(image))

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image", 
                    "image": pil_image  # Use the processed PIL image
                },
                {
                    "type": "text", 
                    "text": "Identify the fruit or vegetable in the image and strictly categorize its freshness as one of the following: 'extreme freshness,' 'mild freshness,' or 'worse.' The assessment should be based on observable factors such as color, texture, and signs of spoilage. Make sure to return only one of these three categories in your response, along with a brief explanation of why this category was chosen."
                }   
            ]
        }
    ]
    
    # Prepare input for inference
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output from the Qwen model
    generated_ids = model.generate(**inputs, max_new_tokens=64)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # Post-process the output to extract fruit type and freshness
    fruit_type = "unknown"
    freshness = "unknown"

    if "apple" in output_text.lower():
        fruit_type = "apple"
    elif "banana" in output_text.lower():
        fruit_type = "banana"
    elif "potato" in output_text.lower():
        fruit_type = "potato"
    elif "orange" in output_text.lower():
        fruit_type = "orange"
    elif "guava" in output_text.lower():
        fruit_type = "guava"
    elif "pomegranate" in output_text.lower():
        fruit_type = "pomegranate"
    elif "strawberry" in output_text.lower():
        fruit_type = "strawberry"
    # Add detection logic for other fruits/vegetables here as needed

    if "extreme freshness" in output_text.lower():
        freshness = "extreme freshness"
    elif "mild freshness" in output_text.lower():
        freshness = "mild freshness"
    elif "worse" in output_text.lower():
        freshness = "worse"

    # Get estimated shelf life based on fruit type and freshness
    shelf_life = get_shelf_life(fruit_type, freshness)

    # Get freshness index
    freshness_index = get_freshness_index(freshness)

    # Save data to CSV
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, fruit_type, freshness, freshness_index, shelf_life])

    # Append shelf life and freshness index to the output
    output_text += f"\n\nFreshness Index: {freshness_index}\nEstimated Shelf Life for {fruit_type.capitalize()} with {freshness.capitalize()}: {shelf_life}"

    return output_text

# Gradio UI
def run_gradio_interface(image):
    output_text = process_freshness(image)
    return output_text

# Define Gradio interface with image input and text output
with gr.Blocks() as demo:
    
    # Add a title and description
    gr.Markdown("## Freshness Detection with Shelf Life Estimation")
    gr.Markdown(
        """
        **Upload an image of a fruit or vegetable**, and the Qwen model will identify the item and assess its freshness.
        Based on the freshness level, the app will also estimate the shelf life of the fruit or vegetable.
        The results will be saved in a CSV file for further use.
        """
    )
    
    # Define the layout for the interface
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="numpy", label="Upload Image")  # Image upload box
            submit_button = gr.Button("Run Freshness Detection")  # Submit button
        
        with gr.Column():
            output_box = gr.Textbox(label="Freshness Detection Output", lines=10, max_lines=20)  # Text output

    # Add interaction to trigger inference
    submit_button.click(fn=run_gradio_interface, inputs=image_input, outputs=output_box)

# Launch the app
demo.launch(share=True)
